In [ ]:
# pip install git+https://github.com/lsst-sims/rubin_nights.git
import os
import numpy as np
from astropy.time import Time

from rubin_nights.connections import get_clients, get_access_token
from rubin_nights.consdb_query import ConsDbTap, ConsDbFastAPI, ConsDbSql
from rubin_nights.augment_visits import augment_visits, exclude_visits, fetch_excluded_visits
from rubin_nights.targets_and_visits import targets_and_visits, flag_potential_bad_visits

## Query the ConsDB 

First, set up the location and authentication for the ConsDB.

If you are on an RSP - you're good to go, this will be taken care of automatically for you.
If you are outside an RSP, choose which site you'd like to use and identify your RSP token. 

In [ ]:
on_rsp = False
# Are you on an RSP?
if on_rsp:
    api_base = os.getenv("EXTERNAL_INSTANCE_URL", "")
    token = get_access_token()
    site = None
# Or are you outside of an RSP? - just use USDF and your own USDF-RSP token
# See https://rsp.lsst.io/guides/auth/creating-user-tokens.html
else:
    api_base = "https://usdf-rsp.slac.stanford.edu"
    # Substitute the location of your own tokenfile
    # If you prefer, this will also get token info from an "ACCESS_TOKEN" environment variable
    tokenfile = os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp")
    token = get_access_token(tokenfile)
    site = 'usdf'
    
consdb_tap = ConsDbTap(api_base=api_base, token=token)
consdb_fastapi = ConsDbFastAPI(api_base=api_base, auth=('user', token))

# The ConsDbSql access will only work within the USDF or Summit, but connects using postgres directly 
# and then runs queries through pandas. 
if on_rsp:
    consdb_sql = ConsDbSql(site="usdf")

It's worth noting that the queries below are much faster when run at the RSP, and speeds generally get faster from consdb_sql -> consdb_fastapi -> consdb_tap. However, errors due to sql are easier to find and correct using consdb_tap or possibly consdb_sql than consdb_fastapi, which does not always pass back the proper sql errors (instead just giving a 500 https error). 

General queries are supported and the results can be manipulated as dataframes. See [sdm-schemas.lsst.io](https://sdm-schemas.lsst.io) for more information on available data.

In [ ]:
# Here is a simple query
query = "select * from cdb_lsstcam.visit1 where science_program = 'BLOCK-365' and day_obs=20250621"

In [ ]:
%%time
visits = consdb_fastapi.query(query)
print(len(visits))
print(visits.columns)

In [ ]:
%%time
visits2 = consdb_tap.query(query)
print(len(visits2))
print(visits2.columns)

In [ ]:
np.all(visits['visit_id'] == visits2['visit_id'])

In [ ]:
visits.head()[['visit_id', 'obs_start', 's_ra', 's_dec', 'sky_rotation', 'band', 'target_name', 'observation_reason']]

In [ ]:
visits.groupby(['target_name', 'band']).agg({'obs_start': ('first', 'last', 'count')})

There is also a convenience function to query all visit1 and visit1_quicklook data between two times. This is inherited from the parent class, for any of the Consdb (specific service) class above. 

In [ ]:
t_start = Time("2025-06-20T12:00:00", format='isot', scale='tai')
t_end = Time("2025-06-21T12:00:00", format='isot', scale='tai')

visits = consdb_fastapi.get_visits("lsstcam", t_start, t_end, visit_constraint="science_program = 'BLOCK-365'", augment=False)
print(len(visits))
visits.head()

## Add extra computed values 

There is a function to add some additional useful columns, such as the predicted zeropoint (which can be used to estimate cloud extinction) or moon separation, along with additional values useful to match the opsim schema.

In [ ]:
visits_plus = augment_visits(visits)
# What columns are added?
set(visits_plus.columns) - set(visits.columns)

## Removing bad visits

DM maintains a repo at [excluded_visits](https://github.com/lsst-dm/excluded_visits) to track known bad visits, and you can automatically download bad visits from this repo.

In [ ]:
bad_visit_ids = fetch_excluded_visits(instrument="lsstcam")
bad_visit_ids = [b for b in bad_visit_ids if b in visits.visit_id.values]
print(np.sort(bad_visit_ids))
better_visits = exclude_visits(visits=visits, bad_visit_ids=bad_visit_ids)
print(len(visits), len(better_visits))

## Targets and Visits 

The `lsst.sal.Scheduler.logevent_target` and `lsst.sal.Scheduler.logevent_observation`, coupled with `lsst.sal.Scheduler.logevent_nextvisit` can provide additional useful information, including the salIndex of the script which generated the observation. These can be retrieve with `targets_and_visits`. 

Potential bad visits can also be marked by the lack of an `observation` event following the `target` (indicating the script failed somewhere), the lack of `quicklook` values (indicating the image failed processing, although it can also just indicate that rapid analysis was unable to write to the ConsDB .. plus there are many reasons why a visit might fail processing), or a large offset between the expected zeropoint and the measured zeropoint for the image. This can be useful to flag some suspect visits, but does not identify all bad visits, nor are all visits thus-identified actually bad.

In [ ]:
# This requires querying the EFD as well as the ConsDB
endpoints = get_clients(site=site, auth_token=token)
target_visits, cols, target_obs, nv, v = targets_and_visits(t_start, t_end, endpoints, queue_index=1)

In [ ]:
target_visits.head()[['time_target', 'time_nextvisit', 'time_observation', 'scriptSalIndex', 'groupId','obs_start', 'observation_reason',  'target_name', 'band']]

In [ ]:
# Flag bad visits with 1.5 magnitudes or more of extinction and if there is no quicklook values
flagged_visit_ids = flag_potential_bad_visits(target_visits,  extinction = 1.5, no_quicklook = True)
print(np.sort(flagged_visit_ids))
better_visits = exclude_visits(visits=visits, bad_visit_ids=flagged_visit_ids)
print(len(visits), len(better_visits))